Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Get the Number of Published Models

This example returns the number of projects with published models and the total number of published models by destination type.

In [ ]:
import requests
import os, json
import pprint
import getpass

# Specify the URL for your host server
hostport="http://myserver.com"

In [ ]:
# Specify the user ID and password for a non-admin user.
user="<userID>"
password="<password>"

In [ ]:
# Specify the user ID and password for the administrator user.
user="<admin_userID>"
password="<password>"

In [ ]:
# Get authentication token.
authUri="/SASLogon/oauth/token"

headersAuth={
    "accept":"application/json",
    "content-type":"application/x-www-form-urlencoded",
    "Authorization":"Basic c2FzLmVjOg=="
}
authToken=""
user=""
password=""
notAuthed=True

while notAuthed :
    user = input("Enter user ID:")
    password = getpass.getpass('Enter password for user %s:' % user)
    authBody='grant_type=password&username=' + user + '&password=' + password
    authReturn = requests.post(hostport+authUri, data=authBody, headers=headersAuth)
    print(authReturn)
    if authReturn.status_code == requests.codes.ok :
        authToken = authReturn.json()['access_token']
        notAuthed = False
    else :
        print("Please enter the correct user ID and password.")
    
password = ""
#print(authToken)

In [ ]:
headersGet = {
    'Authorization': 'Bearer ' + authToken
}

In [ ]:
models = requests.get(hostport + "/modelPublish/models", headers=headersGet)
print(models)
print(models.json()["count"])
#pprint.pprint(models.json())

In [ ]:
# Retrieves the unique model count.
def getModelCount(models) :
    #create a dictionary
    uniqueModel = {}
    items = models.json()["items"]
    for item in items :
        #print(item['modelId'])
        # only count completed publish model
        if item['state'] == 'completed':
            if item['modelId'] not in uniqueModel :
                #print(item['modelId'] + "exist")
                uniqueModel[item['modelId']] = item['publishName']
    return len(uniqueModel)

def getCompletedModelCount(models) :
    #create a dictionary
    completedModel = {}
    items = models.json()["items"]
    for item in items :
        # Print(item['modelId'])
        # Only counts completed publish models
        if item['state'] == 'completed':
            completedModel[item['id']] = item['publishName']
    return len(completedModel)

In [ ]:
# Prints the unique published model count.
modelCount = getModelCount(models)
print(modelCount)

# Print completed published models count.
modelCount = getCompletedModelCount(models)
print(modelCount)

In [ ]:
def getCompletedProjectModelCount(models) :
    # Creates a dictionary
    completedModel = {}
    items = models.json()["items"]
    for item in items :
        # Pprint(item['modelId'])
        # Only counts completed publish models
        if item['state'] == 'completed':
            # Check if the projectId field is in the item or not
            if 'projectId' in item :
                if item['projectId'] not in completedModel :
                    completedModel[item['projectId']] = item['publishName']
            else :
                if item['modelId'] not in completedModel :
                    completedModel[item['modelId']] = item['publishName'] 
                    
    return len(completedModel)

In [ ]:
# Prints completed published models count
modelCount = getCompletedProjectModelCount(models)
print(modelCount)

## Get Models by Publish Type

In [ ]:
# Here you can use the publishType or type. There are four types that can be specified.
filter="?filter=eq(type, \"mas\")"
#filter="?filter=eq(type, \"ep\")"
#filter="?filter=eq(type, \"casModel\")"
#filter="?filter=eq(type, \"container\")"
models = requests.get(hostport + "/modelPublish/models"+filter, headers=headersGet)
print(models)
#print(models.json())
print(models.json()["count"])
#pprint.pprint(models.json())


## Get Models by Destination Name

In [ ]:
# The destinationName parameter is same as the targetServerName parameter.
filter="?filter=eq(targetServerName, \"maslocal\")"
#filter="?filter=eq(targetServerName, \"casWB\")"
models = requests.get(hostport + "/modelPublish/models"+filter, headers=headersGet)
print(models)
#print(models.json())
print(models.json()["count"])
#pprint.pprint(models.json())

## Get Models by creationTimeStamp

In [ ]:
#filter="?filter=and(eq(destinationName, maslocal),gt(creationTimeStamp, 2019-12-05T00:00:00.000Z))"
filter="?filter=lt(creationTimeStamp, \"2019-12-17\")"
models = requests.get(hostport + "/modelPublish/models"+filter, headers=headersGet)
print(models)
#print(models.json())
print(models.json()["count"])
#pprint.pprint(models.json())

In [ ]:
filter="?filter=gt(creationTimeStamp, \"2019-11-17\")"  #recent a month
#filter="?filter=gt(creationTimeStamp, \"2019-06-17\")"  #recent 6 month
#filter="?filter=lt(creationTimeStamp, \"2018-12-17\")"  #recent a year
models = requests.get(hostport + "/modelPublish/models"+filter, headers=headersGet)
print(models)
#print(models.json())
print(models.json()["count"])
#pprint.pprint(models.json())

In [ ]:
# find model by modelId
modelId="bf70131d-35c3-47ce-80e3-8487976d78ec"
filter="?filter=eq(modelId,\""+ modelId + "\")"
#filter="?filter=eq(modelUUID,\""+ modelId + "\")"

models = requests.get(hostport + "/modelPublish/models"+filter, headers=headersGet)
print(models)
pprint.pprint(models.json())
print(models.json()["count"])

In [ ]:
# find model by sourceUri
sourceUri="/modelRepository/models/bf70131d-35c3-47ce-80e3-8487976d78ec"
filter="?filter=eq(sourceUri,\""+ sourceUri + "\")"

models = requests.get(hostport + "/modelPublish/models"+filter, headers=headersGet)
print(models)
pprint.pprint(models.json())
print(models.json()["count"])

## Counts Models by Destination Type

In [ ]:
casModelCount=0
masModelCount=0

for i, model in enumerate(models.json()["items"]):
    if "cas"== model["destination"]["destinationType"]:
        casModelCount = casModelCount + 1
    elif "microAnalyticService" == model["destination"]["destinationType"] :
        masModelCount = masModelCount + 1
        
#print(casModelCount)
#print(masModelCount)

## Display Models in a Table

In [ ]:
import pandas as pd

# intialise data of lists.
data = {'Destination Type':['CAS', 'MAS'],
        'Model Count':[casModelCount, masModelCount]}

# Create DataFrame
df = pd.DataFrame(data)
 
# Print the output.
print(df)

In [ ]:
import datetime

cDate = datetime.datetime.now()

cyear = cDate.year
cmonth = cDate.month
cday = cDate.day

year6 = cyear
month6 = cmonth - 6
day6 = cday

if month6 <= 0 : 
    month6 = 12 + month6
    year6 = year6 -1
    
print(year6)
print(month6)
print(day6)

